In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg
import torch.optim as optim
import os

from PIL import Image

import matplotlib.pyplot as plt

def song_to_spectrogram(song_path, length_sec=30, save_image=True, save_name="spectrogram.png"):
    y, sr = librosa.load(song_path)
    y = y[:sr * length_sec]

    S = librosa.feature.melspectrogram(y=y, sr=sr, hop_length=512)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)

    desired_width = 336

    dpi = int(desired_width / plt.figure(figsize=(desired_width / 80, 4)).get_figwidth())
    plt.close('all')

    width = desired_width / dpi
    fig = plt.figure(figsize=(width, width / 3 * 2))
    librosa.display.specshow(S_DB, sr=sr, hop_length=512, x_axis='time', y_axis='mel')
    plt.gca().set_axis_off()

    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    image_array = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    image_array = image_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    image = Image.fromarray(image_array)

    new_size = (432, 288)

    resized_image = image.resize(new_size, Image.LANCZOS)
    resized_image_data = np.array(resized_image)

    if save_image:
        resized_image.save(save_name)
        print("Spectrogram saved at " + save_name)

    plt.close(fig)

    return resized_image_data


In [2]:
def dig(path, ftype):
    """
    returns list of all files of the specified extension in every recursive subdirectory in a given parent dir
    """
    paths = []
    for dirpath, _, files in os.walk(path):
        paths += [(dirpath + "/" + f) for f in files if f.endswith(ftype)]
    return paths

In [3]:
audio_paths = dig("./DEAM_audio", "mp3")

In [5]:
for audio_path in audio_paths:
    if not os.path.exists(f"./DEAM_img/{audio_path[13:-4]}.png"):
        _ = song_to_spectrogram(audio_path, 46, save_name=f"./DEAM_img/{audio_path[13:-4]}.png")